In [1]:
import spotipy
import spotipy.util as util
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

In [2]:
USERNAME = '11124576993'
# full authorization scopes: https://developer.spotify.com/documentation/general/guides/scopes/
SCOPE = 'user-read-recently-played user-top-read user-library-modify user-library-read playlist-read-private playlist-modify-public playlist-modify-private playlist-read-collaborative user-read-email user-read-birthdate user-read-private user-read-playback-state user-modify-playback-state user-read-currently-playing app-remote-control streaming user-follow-read user-follow-modify'
CLIENT_ID = '0e46da03f45b4cd78495b0d520201a77'
CLIENT_SECRET = 'e7bb14e6f6a64547bcd60810ae334f6a'
REDIRECT_URL = 'http://localhost:8888'

In [3]:
# generate token with SCOPE and credentials
token = util.prompt_for_user_token(USERNAME, SCOPE, CLIENT_ID, CLIENT_SECRET, REDIRECT_URL)
if token:
    sp = spotipy.Spotify(auth=token)
else:
    print("The token could not be generated.")

In [4]:
dict_of_user_playlist = sp.current_user_playlists()
total_of_playlists = dict_of_user_playlist['total']

In [5]:
# musical taste of user
def parse_data_from_user_playlists(dict_of_user_playlist):
    '''
    Returns a dictionary of playlist id (as keys) and name, total tracks on playlist (as values)
    Parameters:
        - dict_of_user_playlist - dictionary generated based on the SpotPy API function 'current_user_playlist()'
    '''
    dict_of_playlist_ids = {}
    list_of_name_and_total_tracks = []
    for key_playlists_data, value_playlists_data in dict_of_user_playlist.items():
        if key_playlists_data == 'items':
            for item_playlist in value_playlists_data:
                 for key_playlist, value_playlist in item_playlist.items():
                        if key_playlist == 'id':
                            key_playlist_id = value_playlist
                            dict_of_playlist_ids[key_playlist_id] = ''
#                         if key_playlist == 'name':
#                             list_of_name_and_total_tracks.append(value_playlist)
                        if key_playlist == 'tracks':
                            for key_track, value_track in value_playlist.items():
                                 if key_track == 'total':
                                        list_of_name_and_total_tracks.append(value_track)
                                        dict_of_playlist_ids[key_playlist_id] = list_of_name_and_total_tracks
                                        list_of_name_and_total_tracks = []
    return dict_of_playlist_ids

In [18]:
dict_of_playlist_ids = parse_data_from_user_playlists(dict_of_user_playlist)
df_dict_of_playlist_ids = pd.DataFrame(dict_of_playlist_ids)
df_dict_of_playlist_ids = df_dict_of_playlist_ids.T
df_dict_of_playlist_ids.reset_index(level=0, inplace=True)
df_dict_of_playlist_ids.columns = ['playlist_id', 'total_tracks']
df_dict_of_playlist_ids.head()

,playlist_id,total_tracks
0,4gODJgZoVDYr0M8Ekxntla,471
1,37i9dQZF1DX0u2K70Zsmro,65
2,37i9dQZF1CAik9XcRkiLD5,50
3,5wZSKauJj5UrTxaKnECZN2,44
4,5kLfYTXSHAFVN2Z2L1boKZ,16


In [19]:
# musical taste of user
def parse_data_from_tracks(dict_of_playlist_ids):
    '''
    Returns a dictionary of track id (as keys) and explicit, name, popularity (as values)
    Explicit - boolean value; False - track is uncensored; True - track is censored
    Parameters:
        - dict_of_playlists_id - dictionary generated based on the 'parse_data_from_user_playlists()' function
    '''
    dict_of_track_ids = {}
    list_of_name_popularity_explict = []
    for elem in dict_of_playlist_ids:
        dict_of_user_playlist_tracks = sp.user_playlist_tracks(USERNAME, elem)
        for key_playlist_tracks_data, value_playlist_tracks_data in dict_of_user_playlist_tracks.items():
            if key_playlist_tracks_data == 'items':
                for idx, item in enumerate(value_playlist_tracks_data):
                    for key_playlist, item_playlist in item.items():
                        if key_playlist == 'track':
                            for key_track, value_track in item_playlist.items():
                                if key_track == 'explicit':
                                    list_of_name_popularity_explict.append(elem)
                                    list_of_name_popularity_explict.append(value_track)
                                if key_track == 'id':
                                    key_track_id = value_track
                                    dict_of_track_ids[key_track_id] = ''
#                                 if key_track == 'name':
#                                     list_of_name_popularity_explict.append(value_track)                            
                                if key_track == 'popularity':
                                    list_of_name_popularity_explict.append(value_track)
                                    dict_of_track_ids[key_track_id] = list_of_name_popularity_explict
                                    list_of_name_popularity_explict = []
    return dict_of_track_ids

In [20]:
dict_of_track_ids = parse_data_from_tracks(dict_of_playlist_ids)
df_dict_of_track_ids = pd.DataFrame(dict_of_track_ids)
df_dict_of_track_ids = df_dict_of_track_ids.T
df_dict_of_track_ids.reset_index(level=0, inplace=True)
df_dict_of_track_ids.columns = ['track_id', 'playlist_id', 'explicit', 'track_popularity']
df_dict_of_track_ids.head()

,track_id,playlist_id,explicit,track_popularity
0,0ew27xRdxSexrWbODuLfeE,4gODJgZoVDYr0M8Ekxntla,False,68
1,2Foc5Q5nqNiosCNqttzHof,4gODJgZoVDYr0M8Ekxntla,False,76
2,49FEOyx3GsIxuZSoGdAHvN,4gODJgZoVDYr0M8Ekxntla,False,65
3,3h8vSAehx9i8I4r1zlqSty,4gODJgZoVDYr0M8Ekxntla,False,29
4,27rdGxbavYJeBphck5MZAF,4gODJgZoVDYr0M8Ekxntla,False,91


In [21]:
# maybe used for training dataset...
lst = ['6wWVKhxIU2cEi0K81v7HvP', '2ye2Wgw4gimLv2eAKyk1NB']
res_lst = []
dict_of_artsits_general_info = {}
for idx, value in enumerate(lst):
    dict_of_artist = sp.artist(value)
    dict_of_artsits_general_info[value] = ''
    for key, elem in dict_of_artist.items():
        if key == 'followers':
            for key_followers, value_followers in elem.items():
                if key_followers == 'total':
                    res_lst.append(value_followers)     
        if key == 'genres':
            res_lst.append(elem)
        if key == 'name':
            res_lst.append(elem)
        if key == 'popularity':
            res_lst.append(elem)
            dict_of_artsits_general_info[value] = res_lst
            res_lst = []

In [22]:
# maybe used for training dataset...
df_training_dataset = pd.DataFrame(dict_of_artsits_general_info).T
df_training_dataset.reset_index(level=0, inplace=True)
df_training_dataset.columns = ['artist_id', 'total_followers', 'genres_list', 'artist_name', 'artist_popularity']
df_training_dataset

,artist_id,total_followers,genres_list,artist_name,artist_popularity
0,6wWVKhxIU2cEi0K81v7HvP,2640470,"[alternative metal, german metal, industrial, ...",Rammstein,79
1,2ye2Wgw4gimLv2eAKyk1NB,9994262,"[hard rock, metal, old school thrash, rock, sp...",Metallica,84


In [23]:
list_of_artists = ['6wWVKhxIU2cEi0K81v7HvP', '2ye2Wgw4gimLv2eAKyk1NB']
def assign_albums_to_artist(list_of_artists):
    '''
    Returns a DataFrame with artist ID, album ID, release date of album and total tracks on album
    Parameters:
        - list_of_artists - list with any artist ID's
    '''
    dict_of_basic_artist_albums = {}
    dict_of_albums_info = {}
    list_of_albums = []
    list_of_album_info = []
    for idx_artist, value_artist in enumerate(list_of_artists):
        dict_of_artist_albums = sp.artist_albums(value_artist)
        dict_of_basic_artist_albums[value_artist] = ''
        for key_of_artist_albums_data, value_artist_albums_data in dict_of_artist_albums.items():
            if key_of_artist_albums_data == 'items':
                for idx_album, value_album in enumerate(value_artist_albums_data):
                    for key_album, item_album in value_album.items():
                        if key_album == 'id':
                            list_of_albums.append(item_album)
                            dict_of_basic_artist_albums[value_artist] = list_of_albums
                            key_track_id = item_album
                            dict_of_albums_info[key_track_id] = ''
#                         if key_album == 'name':
#                             list_of_album_info.append(item_album)
                        if key_album == 'release_date':
                            list_of_album_info.append(item_album)
                        if key_album == 'total_tracks':
                            list_of_album_info.append(item_album)
                            dict_of_albums_info[key_track_id] = list_of_album_info
                            list_of_album_info = []
        list_of_albums = []
        
        df_dict_of_basic_artist_albums = pd.DataFrame([(key, value) for (key, L) in dict_of_basic_artist_albums.items() for value in L], 
        columns=['artist_id', 'album_id'])
        
        df_dict_of_albums_info = pd.DataFrame(dict_of_albums_info)
        df_dict_of_albums_info = df_dict_of_albums_info.T
        df_dict_of_albums_info.reset_index(level=0, inplace=True)
        df_dict_of_albums_info.columns = ['album_id', 'release_date', 'total_tracks']
        
        df_artist_albums = pd.merge(df_dict_of_basic_artist_albums, df_dict_of_albums_info, on='album_id')
    return df_artist_albums
df_artist_albums = assign_albums_to_artist(list_of_artists)
df_artist_albums.head()

,artist_id,album_id,release_date,total_tracks
0,6wWVKhxIU2cEi0K81v7HvP,2SU0P5F7ZSN36pUZeYgAAG,2017-05-19,22
1,6wWVKhxIU2cEi0K81v7HvP,6rK4qSVdrNCNnQwRCEFsw9,2016-11-11,13
2,6wWVKhxIU2cEi0K81v7HvP,0o0QM3HLgDRQQslL2ott2a,2015-04-23,12
3,6wWVKhxIU2cEi0K81v7HvP,20mEbRzpqGIWyLtUO6LbjB,2011-12-02,33
4,6wWVKhxIU2cEi0K81v7HvP,29J0MchD0OWNeiMNbPDwfa,2009-10-16,16


In [24]:
def assign_tracks_to_album(df_artist_albums):
    '''
    Returns a DataFrame with album ID and track ID
    Parameters:
        - df_artist_albums - DataFrame generated based on the 'assign_albums_to_artist()' function
    '''
    dict_of_album_tracks = {}
    list_of_track_id = []
#     list_of_track_name = []
    for index, row in assign_albums_to_artist(list_of_artists).iterrows():
        dict_with_album_tracks = sp.album_tracks(row['album_id'])
        dict_of_album_tracks[row['album_id']] = ''
        for key_of_album_tracks_data, value_album_tracks_data in dict_with_album_tracks.items():
            if key_of_album_tracks_data == 'items':
                for idx_track, value_track in enumerate(value_album_tracks_data):
                    for key_track, item_track in value_track.items():
                        if key_track == 'id':
                            list_of_track_id.append(item_track)
#                         if key_track == 'name':
#                             list_of_track_name.append(item_track)
                dict_of_album_tracks[row['album_id']] = list_of_track_id
                list_of_track_id = []
#                 list_of_track_name = []

    df_dict_of_album_tracks = pd.DataFrame([(key, value) for (key, L) in dict_of_album_tracks.items() for value in L], 
    columns=['album_id', 'track_id'])
    return df_dict_of_album_tracks
df_dict_of_album_tracks = assign_tracks_to_album(df_artist_albums)
df_dict_of_album_tracks.head()

,album_id,track_id
0,2SU0P5F7ZSN36pUZeYgAAG,4bOH04HnrNuQTY1vj0RFO1
1,2SU0P5F7ZSN36pUZeYgAAG,4eUGuSFHlY7hdQxy99Klb1
2,2SU0P5F7ZSN36pUZeYgAAG,26D2Sv7cuoPLmMmT9LuFFC
3,2SU0P5F7ZSN36pUZeYgAAG,0kzP8mrolprWH4HOlzdoNm
4,2SU0P5F7ZSN36pUZeYgAAG,4TUzUUKS7HhZyAD35vgqVn


In [25]:
df_artist_album_track = pd.merge(df_artist_albums, df_dict_of_album_tracks, on='album_id')
df_artist_album_track.head()

,artist_id,album_id,release_date,total_tracks,track_id
0,6wWVKhxIU2cEi0K81v7HvP,2SU0P5F7ZSN36pUZeYgAAG,2017-05-19,22,4bOH04HnrNuQTY1vj0RFO1
1,6wWVKhxIU2cEi0K81v7HvP,2SU0P5F7ZSN36pUZeYgAAG,2017-05-19,22,4eUGuSFHlY7hdQxy99Klb1
2,6wWVKhxIU2cEi0K81v7HvP,2SU0P5F7ZSN36pUZeYgAAG,2017-05-19,22,26D2Sv7cuoPLmMmT9LuFFC
3,6wWVKhxIU2cEi0K81v7HvP,2SU0P5F7ZSN36pUZeYgAAG,2017-05-19,22,0kzP8mrolprWH4HOlzdoNm
4,6wWVKhxIU2cEi0K81v7HvP,2SU0P5F7ZSN36pUZeYgAAG,2017-05-19,22,4TUzUUKS7HhZyAD35vgqVn


In [26]:
def assign_track_details(df_dict_of_album_tracks):
    lst_track_info = []
    lst_item_track_info = []
    for idx, row in df_dict_of_album_tracks.iterrows():
        lst = sp.audio_features(row['track_id'])
        dict_track_info = lst[0]
        lst_item_track_info.append(row['track_id'])
        for key, item in dict_track_info.items():
            if key == 'danceability':
                lst_item_track_info.append(item)
            if key == 'energy':
                lst_item_track_info.append(item)
            if key == 'key':
                lst_item_track_info.append(item)
            if key == 'loudness':
                lst_item_track_info.append(item)
            if key == 'mode':
                lst_item_track_info.append(item)
            if key == 'speechiness':
                lst_item_track_info.append(item)
            if key == 'acousticness':
                lst_item_track_info.append(item)
            if key == 'instrumentalness':
                lst_item_track_info.append(item)
            if key == 'liveness':
                lst_item_track_info.append(item)    
            if key == 'valence':
                lst_item_track_info.append(item)
            if key == 'tempo':
                lst_item_track_info.append(item)
            if key == 'duration_ms':
                lst_item_track_info.append(item)
                lst_track_info.append(lst_item_track_info)
                lst_item_track_info = []     
        df_track_info = pd.DataFrame(lst_track_info)
        df_track_info.columns=['track_id', 'danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'duration_ms']
    return df_track_info
df_track_details = assign_track_details(df_dict_of_album_tracks)
df_track_details

,track_id,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms
0,4bOH04HnrNuQTY1vj0RFO1,0.152,0.5090,0,-15.423,1,0.0639,0.006660,0.794000,0.9220,0.1500,75.018,254013
1,4eUGuSFHlY7hdQxy99Klb1,0.363,0.8990,2,-8.188,1,0.1250,0.000257,0.135000,0.8840,0.1240,75.053,285067
2,26D2Sv7cuoPLmMmT9LuFFC,0.520,0.9570,7,-7.298,1,0.0394,0.007630,0.594000,0.4410,0.2240,130.035,306480
3,0kzP8mrolprWH4HOlzdoNm,0.233,0.9360,8,-7.573,1,0.0719,0.007490,0.164000,0.7390,0.1450,138.131,248267
4,4TUzUUKS7HhZyAD35vgqVn,0.513,0.9580,2,-7.734,1,0.0582,0.010400,0.239000,0.9740,0.3450,140.064,266693
5,43L94GIXyy3eWROPhHE7Cu,0.403,0.9740,2,-8.039,1,0.1160,0.007320,0.037900,0.9750,0.2620,169.978,245360
6,59hleeRytt12eFVeFMfGyO,0.475,0.9740,8,-7.890,1,0.0833,0.001620,0.052900,0.9950,0.1410,95.012,214453
7,5sxJ1H0zdjNMPHmH8It9Zc,0.360,0.6970,4,-9.089,1,0.0512,0.145000,0.004240,0.9790,0.1160,115.865,320533
8,249cac1oZNAy4cnrHE25vc,0.369,0.8820,7,-9.340,1,0.0964,0.004530,0.118000,0.9960,0.0597,88.028,466667
9,6Ln9SSA1jnTEGBWIqXU90h,0.480,0.8870,7,-8.494,0,0.0659,0.006680,0.245000,0.9780,0.1310,111.972,328373


In [17]:
df_artist_album_track_details = pd.merge(df_artist_album_track, df_track_details, on='track_id')
df_artist_album_track_details

,artist_id,album_id,release_date,total_of_tracks,track_id,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms
0,6wWVKhxIU2cEi0K81v7HvP,2SU0P5F7ZSN36pUZeYgAAG,2017-05-19,22,4bOH04HnrNuQTY1vj0RFO1,0.152,0.5090,0,-15.423,1,0.0639,0.006660,0.794000,0.9220,0.1500,75.018,254013
1,6wWVKhxIU2cEi0K81v7HvP,2SU0P5F7ZSN36pUZeYgAAG,2017-05-19,22,4eUGuSFHlY7hdQxy99Klb1,0.363,0.8990,2,-8.188,1,0.1250,0.000257,0.135000,0.8840,0.1240,75.053,285067
2,6wWVKhxIU2cEi0K81v7HvP,2SU0P5F7ZSN36pUZeYgAAG,2017-05-19,22,26D2Sv7cuoPLmMmT9LuFFC,0.520,0.9570,7,-7.298,1,0.0394,0.007630,0.594000,0.4410,0.2240,130.035,306480
3,6wWVKhxIU2cEi0K81v7HvP,2SU0P5F7ZSN36pUZeYgAAG,2017-05-19,22,0kzP8mrolprWH4HOlzdoNm,0.233,0.9360,8,-7.573,1,0.0719,0.007490,0.164000,0.7390,0.1450,138.131,248267
4,6wWVKhxIU2cEi0K81v7HvP,2SU0P5F7ZSN36pUZeYgAAG,2017-05-19,22,4TUzUUKS7HhZyAD35vgqVn,0.513,0.9580,2,-7.734,1,0.0582,0.010400,0.239000,0.9740,0.3450,140.064,266693
5,6wWVKhxIU2cEi0K81v7HvP,2SU0P5F7ZSN36pUZeYgAAG,2017-05-19,22,43L94GIXyy3eWROPhHE7Cu,0.403,0.9740,2,-8.039,1,0.1160,0.007320,0.037900,0.9750,0.2620,169.978,245360
6,6wWVKhxIU2cEi0K81v7HvP,2SU0P5F7ZSN36pUZeYgAAG,2017-05-19,22,59hleeRytt12eFVeFMfGyO,0.475,0.9740,8,-7.890,1,0.0833,0.001620,0.052900,0.9950,0.1410,95.012,214453
7,6wWVKhxIU2cEi0K81v7HvP,2SU0P5F7ZSN36pUZeYgAAG,2017-05-19,22,5sxJ1H0zdjNMPHmH8It9Zc,0.360,0.6970,4,-9.089,1,0.0512,0.145000,0.004240,0.9790,0.1160,115.865,320533
8,6wWVKhxIU2cEi0K81v7HvP,2SU0P5F7ZSN36pUZeYgAAG,2017-05-19,22,249cac1oZNAy4cnrHE25vc,0.369,0.8820,7,-9.340,1,0.0964,0.004530,0.118000,0.9960,0.0597,88.028,466667
9,6wWVKhxIU2cEi0K81v7HvP,2SU0P5F7ZSN36pUZeYgAAG,2017-05-19,22,6Ln9SSA1jnTEGBWIqXU90h,0.480,0.8870,7,-8.494,0,0.0659,0.006680,0.245000,0.9780,0.1310,111.972,328373
